## Лабораторная работа №3: Проведение исследований с решающим деревом

In [4]:
!pip install scikit-learn numpy pandas matplotlib seaborn

In [5]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [6]:
date_fruit_path = "/kaggle/input/date-fruit-datasets"

print("Path to Date Fruit dataset files:", date_fruit_path)

# Проверка содержимого папки
if os.path.exists(date_fruit_path):
    files = os.listdir(date_fruit_path)
    print("Files:", files)
else:
    print("Not found")

Path to Date Fruit dataset files: /kaggle/input/date-fruit-datasets
Files: ['Date_Fruit_Datasets']


In [7]:
# Чтение Excel-файла
date_fruit_data = pd.read_excel(f"{date_fruit_path}/Date_Fruit_Datasets/Date_Fruit_Datasets.xlsx")

# Проверка данных
date_fruit_data.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


In [8]:
concrete_strength_path = "/kaggle/input/concrete-compressive-strength"

print("Путь до датасета Concrete Compressive Strength:", concrete_strength_path)

# Проверка содержимого папки
if os.path.exists(concrete_strength_path):
    files = os.listdir(concrete_strength_path)
    print("Содержание:", files)
else:
    print("Не найден")

Путь до датасета Concrete Compressive Strength: /kaggle/input/concrete-compressive-strength
Содержание: ['Concrete Compressive Strength.csv']


In [9]:
# Чтение CSV-файла
concrete_data = pd.read_csv(f"{concrete_strength_path}/Concrete Compressive Strength.csv")

# Проверка данных
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age (day),Concrete compressive strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.986111
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.887366
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.269535
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.052780
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.296075


### 2. Создание бейзлайна и оценка качества

In [10]:
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error, r2_score, make_scorer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder
from sklearn.pipeline import Pipeline


In [11]:
# Разделение на признаки и целевую переменную
X_class = date_fruit_data.drop(columns=['Class'])
y_class = date_fruit_data['Class']

# Разделение на обучающую и тестовую выборки
X_train_class, X_test_class, y_train_class, y_test_class = train_test_split(
    X_class, y_class, test_size=0.2, random_state=42, stratify=y_class
)

# Преобразование целевой переменной
label_encoder = LabelEncoder()
y_train_class = label_encoder.fit_transform(y_train_class)
y_test_class = label_encoder.transform(y_test_class)

Аналогично разделим датасет для регрессии

In [12]:
# Разделение на признаки и целевую переменную
X_reg = concrete_data.drop(columns=['Concrete compressive strength '])
y_reg = concrete_data['Concrete compressive strength ']

# Разделение на обучающую и тестовую выборки
X_train_reg, X_test_reg, y_train_reg, y_test_reg = train_test_split(
    X_reg, y_reg, test_size=0.2, random_state=42
)

Обучим модели для классификации и регрессии из Sklearn и оценим их

In [13]:
tree_classifier = DecisionTreeClassifier(max_depth=5, random_state=42)
tree_classifier.fit(X_train_class, y_train_class)

y_pred_class = tree_classifier.predict(X_test_class)

accuracy_tree = accuracy_score(y_test_class, y_pred_class)
f1_tree = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Decision Tree Classifier - Accuracy: {accuracy_tree:.4f}, F1-Score: {f1_tree:.4f}")

Decision Tree Classifier - Accuracy: 0.8056, F1-Score: 0.7969


In [14]:
tree_regressor = DecisionTreeRegressor(max_depth=5, random_state=42)
tree_regressor.fit(X_train_reg, y_train_reg)

y_pred_reg = tree_regressor.predict(X_test_reg)

rmse_tree = mean_squared_error(y_test_reg, y_pred_reg, squared=False)
r2_tree = r2_score(y_test_reg, y_pred_reg)

print(f"Decision Tree Regressor - RMSE: {rmse_tree:.4f}, R²: {r2_tree:.4f}")

Decision Tree Regressor - RMSE: 9.5792, R²: 0.6439


Точность для встроенной в Sklearn модели решающего дерева получилась приемлемой (80.5% точности для классификатора, 9.57 RMSE для регрессора). Теперь улучшим бейзлайн и проанализируем результаты.

### 3. Улучшение бейзлайна
Будем перебирать гиперпараметры max_depth, min_samples_split, min_samples_leaf решающего дерева

In [15]:
# 1. Подбор гиперпараметров для DecisionTreeClassifier с расширенным поиском
print("=== Decision Tree Classifier - Hyperparameter Tuning ===")
param_grid_classifier = {
    'max_depth': [2, 3, 5, 7, 10, 15, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'criterion': ['gini', 'entropy', 'log_loss'],
    'max_features': ['sqrt', 'log2', None]
}

grid_search_classifier = GridSearchCV(
    estimator=DecisionTreeClassifier(random_state=42),
    param_grid=param_grid_classifier,
    scoring={'f1_weighted': make_scorer(f1_score, average='weighted'), 
             'accuracy': 'accuracy',
             # 'precision_weighted': make_scorer(precision_score, average='weighted')
            },
    refit='f1_weighted',
    cv=5,
    verbose=1,
    n_jobs=-1,
    return_train_score=True
)


grid_search_classifier.fit(X_train_class, y_train_class)

# Лучшие параметры и результат
best_params_classifier = grid_search_classifier.best_params_
best_score_classifier = grid_search_classifier.best_score_

print(f"\nЛучшие параметры для классификатора: {best_params_classifier}")
print(f"Лучший F1-Score на кросс-валидации: {best_score_classifier:.4f}")

# Анализ переобучения
best_idx = grid_search_classifier.best_index_
train_f1 = grid_search_classifier.cv_results_['mean_train_f1_weighted'][best_idx]
val_f1 = grid_search_classifier.cv_results_['mean_test_f1_weighted'][best_idx]
print(f"Train F1-Score: {train_f1:.4f}, Validation F1-Score: {val_f1:.4f}")
print(f"Разница (train - val): {train_f1 - val_f1:.4f}")

# 2. Подбор гиперпараметров для DecisionTreeRegressor с расширенным поиском
print("\n=== Decision Tree Regressor - Hyperparameter Tuning ===")
param_grid_regressor = {
    'max_depth': [2, 3, 5, 7, 10, 15, None],
    'min_samples_split': [2, 5, 10, 15, 20],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'criterion': ['squared_error', 'friedman_mse', 'absolute_error'],
    'max_features': ['sqrt', 'log2', None, 0.5, 0.8]
}

grid_search_regressor = GridSearchCV(
    estimator=DecisionTreeRegressor(random_state=42),
    param_grid=param_grid_regressor,
    scoring={'neg_rmse': 'neg_root_mean_squared_error',
             'neg_mae': 'neg_mean_absolute_error',
             'r2': 'r2'},
    refit='neg_rmse',
    cv=5,
    verbose=1,
    n_jobs=-1,
    return_train_score=True
)

grid_search_regressor.fit(X_train_reg, y_train_reg)

# Лучшие параметры и результат
best_params_regressor = grid_search_regressor.best_params_
best_score_regressor = -grid_search_regressor.best_score_

print(f"\nЛучшие параметры для регрессора: {best_params_regressor}")
print(f"Лучший RMSE на кросс-валидации: {best_score_regressor:.4f}")

# Дополнительная информация о лучшей модели регрессора
best_idx_reg = grid_search_regressor.best_index_
train_rmse = -grid_search_regressor.cv_results_['mean_train_neg_rmse'][best_idx_reg]
val_rmse = -grid_search_regressor.cv_results_['mean_test_neg_rmse'][best_idx_reg]
r2_score_val = grid_search_regressor.cv_results_['mean_test_r2'][best_idx_reg]

print(f"Train RMSE: {train_rmse:.4f}, Validation RMSE: {val_rmse:.4f}")
print(f"Validation R²: {r2_score_val:.4f}")
print(f"Разница RMSE (train - val): {train_rmse - val_rmse:.4f}")

# Вывод топ-3 параметров для каждой модели
print("\n=== Топ-3 параметров для классификатора ===")
results_classifier = pd.DataFrame(grid_search_classifier.cv_results_)
top_3_classifier = results_classifier.nsmallest(3, 'rank_test_f1_weighted')
for i, (_, row) in enumerate(top_3_classifier.iterrows(), 1):
    print(f"{i}. max_depth={row['param_max_depth']}, "
          f"min_samples_split={row['param_min_samples_split']}, "
          f"min_samples_leaf={row['param_min_samples_leaf']}, "
          f"F1={row['mean_test_f1_weighted']:.4f}")

print("\n=== Топ-3 параметров для регрессора ===")
results_regressor = pd.DataFrame(grid_search_regressor.cv_results_)
top_3_regressor = results_regressor.nsmallest(3, 'rank_test_neg_rmse')
for i, (_, row) in enumerate(top_3_regressor.iterrows(), 1):
    print(f"{i}. max_depth={row['param_max_depth']}, "
          f"min_samples_split={row['param_min_samples_split']}, "
          f"min_samples_leaf={row['param_min_samples_leaf']}, "
          f"RMSE={-row['mean_test_neg_rmse']:.4f}")

=== Decision Tree Classifier - Hyperparameter Tuning ===
Fitting 5 folds for each of 1575 candidates, totalling 7875 fits

Лучшие параметры для классификатора: {'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Лучший F1-Score на кросс-валидации: 0.8342
Train F1-Score: 0.9993, Validation F1-Score: 0.8342
Разница (train - val): 0.1651

=== Decision Tree Regressor - Hyperparameter Tuning ===
Fitting 5 folds for each of 2625 candidates, totalling 13125 fits

Лучшие параметры для регрессора: {'criterion': 'friedman_mse', 'max_depth': 15, 'max_features': 0.8, 'min_samples_leaf': 1, 'min_samples_split': 2}
Лучший RMSE на кросс-валидации: 6.8788
Train RMSE: 0.7426, Validation RMSE: 6.8788
Validation R²: 0.8284
Разница RMSE (train - val): -6.1362

=== Топ-3 параметров для классификатора ===
1. max_depth=10, min_samples_split=2, min_samples_leaf=1, F1=0.8342
2. max_depth=10, min_samples_split=2, min_samples_leaf=1, F1=0.8342
3. max_dep

Обучим модели с лучшими гиперпараметрами

In [16]:
# 1. Обучение DecisionTreeClassifier с лучшими параметрами
print("\n=== Обучение Decision Tree Classifier с лучшими параметрами ===")
best_tree_classifier = DecisionTreeClassifier(
    max_depth=best_params_classifier['max_depth'],
    min_samples_split=best_params_classifier['min_samples_split'],
    min_samples_leaf=best_params_classifier['min_samples_leaf'],
    random_state=42
)

best_tree_classifier.fit(X_train_class, y_train_class)

# Предсказания на тестовой выборке
y_pred_class = best_tree_classifier.predict(X_test_class)

# Оценка качества
accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test F1-Score: {f1:.4f}")

# 2. Обучение DecisionTreeRegressor с лучшими параметрами
print("\n=== Обучение Decision Tree Regressor с лучшими параметрами ===")
best_tree_regressor = DecisionTreeRegressor(
    max_depth=best_params_regressor['max_depth'],
    min_samples_split=best_params_regressor['min_samples_split'],
    min_samples_leaf=best_params_regressor['min_samples_leaf'],
    random_state=42
)

best_tree_regressor.fit(X_train_reg, y_train_reg)

# Предсказания на тестовой выборке
y_pred_reg = best_tree_regressor.predict(X_test_reg)

# Оценка качества
rmse = mean_squared_error(y_test_reg, y_pred_reg, squared=False)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Test RMSE: {rmse:.4f}")
print(f"Test R²: {r2:.4f}")


=== Обучение Decision Tree Classifier с лучшими параметрами ===
Test Accuracy: 0.8556
Test F1-Score: 0.8587

=== Обучение Decision Tree Regressor с лучшими параметрами ===
Test RMSE: 6.8943
Test R²: 0.8155


Точность обоих моделей возросла, причем для регрессора - существенно (RMSE 6.89 против 9.57)

### 4. Имплементация алгоритма машинного обучения
Напишем собственную реализацию решающего дерева для классификации и регрессии, затем обучим модели на тестовых данных и сравним по качеству с реализациями из Sklearn.

In [17]:
class DecisionTreeNode:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None, samples=None):
        self.feature = feature          # Признак для разбиения
        self.threshold = threshold      # Пороговое значение
        self.left = left                # Левое поддерево
        self.right = right              # Правое поддерево
        self.value = value              # Значение в листе
        self.samples = samples          # Количество образцов в узле

class DecisionTreeClassifierCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1, 
                 criterion='gini', max_features=None, random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.max_features = max_features
        self.random_state = random_state
        self.root = None
        self.n_features_ = None
        self.n_classes_ = None
        
        if random_state is not None:
            np.random.seed(random_state)

    def _calculate_impurity(self, y):
        """Вычисление критерия неоднородности"""
        if self.criterion == 'gini':
            proportions = np.bincount(y) / len(y)
            return 1 - np.sum(proportions ** 2)
        elif self.criterion == 'entropy':
            proportions = np.bincount(y) / len(y)
            proportions = proportions[proportions > 0]  # Избегаем log(0)
            return -np.sum(proportions * np.log2(proportions))
        else:
            raise ValueError(f"Неизвестный критерий: {self.criterion}")

    def _split(self, X, y, feature, threshold):
        """Разделение данных по признаку и порогу"""
        left_mask = X[:, feature] < threshold
        right_mask = ~left_mask
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

    def _best_split(self, X, y):
        """Поиск лучшего разбиения с учетом max_features"""
        best_gain = -float('inf')
        best_split = None
        
        # Выбор подмножества признаков для поиска
        n_features = X.shape[1]
        if self.max_features == 'sqrt':
            max_features = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            max_features = int(np.log2(n_features))
        elif isinstance(self.max_features, float):
            max_features = int(self.max_features * n_features)
        elif self.max_features is None:
            max_features = n_features
        else:
            max_features = self.max_features
            
        # Выбор случайного подмножества признаков
        feature_indices = np.random.choice(n_features, min(max_features, n_features), replace=False)
        
        current_impurity = self._calculate_impurity(y)
        
        for feature in feature_indices:
            thresholds = np.unique(X[:, feature])
            
            # Случайная выборка порогов для ускорения
            if len(thresholds) > 20:
                thresholds = np.random.choice(thresholds, 20, replace=False)
                
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
                
                # Проверка минимального количества образцов
                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue
                
                # Вычисление gain
                impurity_left = self._calculate_impurity(y_left)
                impurity_right = self._calculate_impurity(y_right)
                
                n_left, n_right = len(y_left), len(y_right)
                n_total = n_left + n_right
                
                gain = current_impurity - (n_left/n_total * impurity_left + n_right/n_total * impurity_right)
                
                if gain > best_gain:
                    best_gain = gain
                    best_split = (feature, threshold, gain)

        return best_split

    def _build_tree(self, X, y, depth):
        """Рекурсивное построение дерева"""
        n_samples = len(y)
        
        # Условия остановки
        if (depth == self.max_depth or 
            n_samples < self.min_samples_split or 
            len(np.unique(y)) == 1):
            
            # Находим наиболее частый класс
            unique, counts = np.unique(y, return_counts=True)
            value = unique[np.argmax(counts)]
            return DecisionTreeNode(value=value, samples=n_samples)

        # Поиск лучшего разбиения
        split_result = self._best_split(X, y)
        if split_result is None:
            unique, counts = np.unique(y, return_counts=True)
            value = unique[np.argmax(counts)]
            return DecisionTreeNode(value=value, samples=n_samples)
        
        feature, threshold, gain = split_result
        
        # Разделение данных
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        
        # Проверка минимального количества образцов
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            unique, counts = np.unique(y, return_counts=True)
            value = unique[np.argmax(counts)]
            return DecisionTreeNode(value=value, samples=n_samples)
        
        # Рекурсивное построение поддеревьев
        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)
        
        return DecisionTreeNode(feature=feature, threshold=threshold, 
                              left=left_child, right=right_child, samples=n_samples)

    def fit(self, X, y):
        """Обучение модели"""
        self.n_features_ = X.shape[1]
        self.n_classes_ = len(np.unique(y))
        self.root = self._build_tree(np.array(X), np.array(y), 0)
        return self

    def _predict_one(self, x, node):
        """Предсказание для одного образца"""
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

    def predict(self, X):
        """Предсказание для набора данных"""
        return np.array([self._predict_one(x, self.root) for x in X])

    def predict_proba(self, X):
        """Вероятностные предсказания"""
        predictions = []
        for x in X:
            node = self.root
            while node.value is None:
                if x[node.feature] < node.threshold:
                    node = node.left
                else:
                    node = node.right
            
            # Создаем вектор вероятностей
            proba = np.zeros(self.n_classes_)
            proba[int(node.value)] = 1.0
            predictions.append(proba)
        
        return np.array(predictions)


class DecisionTreeRegressorCustom:
    def __init__(self, max_depth=None, min_samples_split=2, min_samples_leaf=1,
                 criterion='mse', max_features=None, random_state=None):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        self.min_samples_leaf = min_samples_leaf
        self.criterion = criterion
        self.max_features = max_features
        self.random_state = random_state
        self.root = None
        self.n_features_ = None
        
        if random_state is not None:
            np.random.seed(random_state)

    def _calculate_loss(self, y):
        """Вычисление функции потерь"""
        if self.criterion == 'mse':
            return np.mean((y - np.mean(y)) ** 2)
        elif self.criterion == 'mae':
            return np.mean(np.abs(y - np.median(y)))
        else:
            raise ValueError(f"Неизвестный критерий: {self.criterion}")

    def _split(self, X, y, feature, threshold):
        """Разделение данных по признаку и порогу"""
        left_mask = X[:, feature] < threshold
        right_mask = ~left_mask
        return X[left_mask], y[left_mask], X[right_mask], y[right_mask]

    def _best_split(self, X, y):
        """Поиск лучшего разбиения"""
        best_gain = -float('inf')
        best_split = None
        
        current_loss = self._calculate_loss(y)
        
        # Выбор подмножества признаков
        n_features = X.shape[1]
        if self.max_features == 'sqrt':
            max_features = int(np.sqrt(n_features))
        elif self.max_features == 'log2':
            max_features = int(np.log2(n_features))
        elif isinstance(self.max_features, float):
            max_features = int(self.max_features * n_features)
        elif self.max_features is None:
            max_features = n_features
        else:
            max_features = self.max_features
            
        feature_indices = np.random.choice(n_features, min(max_features, n_features), replace=False)
        
        for feature in feature_indices:
            thresholds = np.unique(X[:, feature])
            
            # Случайная выборка порогов для ускорения
            if len(thresholds) > 20:
                thresholds = np.random.choice(thresholds, 20, replace=False)
                
            for threshold in thresholds:
                X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
                
                if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
                    continue
                
                loss_left = self._calculate_loss(y_left)
                loss_right = self._calculate_loss(y_right)
                
                n_left, n_right = len(y_left), len(y_right)
                n_total = n_left + n_right
                
                gain = current_loss - (n_left/n_total * loss_left + n_right/n_total * loss_right)
                
                if gain > best_gain:
                    best_gain = gain
                    best_split = (feature, threshold, gain)

        return best_split

    def _build_tree(self, X, y, depth):
        """Рекурсивное построение дерева"""
        n_samples = len(y)
        
        # Условия остановки
        if (depth == self.max_depth or 
            n_samples < self.min_samples_split or 
            self._calculate_loss(y) < 1e-10):
            
            if self.criterion == 'mse':
                value = np.mean(y)
            else:
                value = np.median(y)
            return DecisionTreeNode(value=value, samples=n_samples)

        # Поиск лучшего разбиения
        split_result = self._best_split(X, y)
        if split_result is None:
            if self.criterion == 'mse':
                value = np.mean(y)
            else:
                value = np.median(y)
            return DecisionTreeNode(value=value, samples=n_samples)
        
        feature, threshold, gain = split_result
        
        # Разделение данных
        X_left, y_left, X_right, y_right = self._split(X, y, feature, threshold)
        
        if len(y_left) < self.min_samples_leaf or len(y_right) < self.min_samples_leaf:
            if self.criterion == 'mse':
                value = np.mean(y)
            else:
                value = np.median(y)
            return DecisionTreeNode(value=value, samples=n_samples)
        
        # Рекурсивное построение поддеревьев
        left_child = self._build_tree(X_left, y_left, depth + 1)
        right_child = self._build_tree(X_right, y_right, depth + 1)
        
        return DecisionTreeNode(feature=feature, threshold=threshold, 
                              left=left_child, right=right_child, samples=n_samples)

    def fit(self, X, y):
        """Обучение модели"""
        self.n_features_ = X.shape[1]
        self.root = self._build_tree(np.array(X), np.array(y), 0)
        return self

    def _predict_one(self, x, node):
        """Предсказание для одного образца"""
        if node.value is not None:
            return node.value
        if x[node.feature] < node.threshold:
            return self._predict_one(x, node.left)
        else:
            return self._predict_one(x, node.right)

    def predict(self, X):
        """Предсказание для набора данных"""
        return np.array([self._predict_one(x, self.root) for x in X])

Обучим модели и оценим их качество

In [18]:
scaler = StandardScaler()
X_train_class = scaler.fit_transform(X_train_class)
X_test_class = scaler.transform(X_test_class)

custom_tree_classifier = DecisionTreeClassifierCustom(max_depth=5, min_samples_split=2)
custom_tree_classifier.fit(X_train_class, y_train_class)

y_pred_class = custom_tree_classifier.predict(X_test_class)
accuracy = accuracy_score(y_test_class, y_pred_class)
f1 = f1_score(y_test_class, y_pred_class, average='weighted')

print(f"Custom Decision Tree Classifier - Accuracy: {accuracy:.4f}, F1-Score: {f1:.4f}")
     

Custom Decision Tree Classifier - Accuracy: 0.8389, F1-Score: 0.8429


In [20]:
scaler = StandardScaler()
X_train_reg = scaler.fit_transform(X_train_reg)
X_test_reg = scaler.transform(X_test_reg)

custom_tree_regressor = DecisionTreeRegressorCustom(max_depth=5, min_samples_split=2)
custom_tree_regressor.fit(X_train_reg, np.array(y_train_reg))

y_pred_reg = custom_tree_regressor.predict(X_test_reg)
rmse = mean_squared_error(y_test_reg, y_pred_reg, squared=False)
r2 = r2_score(y_test_reg, y_pred_reg)

print(f"Custom Decision Tree Regressor - RMSE: {rmse:.4f}, R²: {r2:.4f}")
     


Custom Decision Tree Regressor - RMSE: 8.4854, R²: 0.7206


In [ ]:
Точность реализованных вручную моделей близка к библиотечным.